In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv
import shutil

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
import math
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import LeaveOneLabelOut, cross_val_score, permutation_test_score
#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.workflows.fmri.fsl import create_susan_smooth

from nilearn import input_data

%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects_ser.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
sub_list = sub_list[0:3]

In [3]:
all_rois = ["lOFC_fill_ser", "mOFC_fill_ser"]

In [4]:
frames = np.arange(-1, 6) #number of frames to hold onto

In [5]:
##add a column to the timing file coding for the TR in which an event occured
event_file = home_dir + 'data/' + sub + '/design/ser_state.csv'
timing = pd.read_csv(event_file)
timing =  timing.sort(['run','onset'])
tr = 1.5
for run in range(1,4):
    event_time = timing.loc[timing['run']==run,'onset'].values
    tr_index = map(lambda x: int(math.floor(x/tr)),event_time)
    timing.loc[timing['run']==run,'tr_index'] = tr_index

NameError: name 'sub' is not defined

In [165]:
def load_data(sub,roi):
    all_data = {}
    for run in range(1,4):
        #data files
        mask_f = home_dir + 'data/' + sub + '/masks/' + roi + '.nii.gz'
        func_f = home_dir + 'analysis/ser_8mm-PE/' + sub + '/preproc/run_' + str(run) + '/unsmoothed_timeseries.nii.gz'

        #load data and detrend
        nifti_masker = input_data.NiftiMasker(mask_img = mask_f, standardize = True)
        fmri_masked = nifti_masker.fit_transform(func_f)
        
        all_data['run'+str(run)] = fmri_masked
    return all_data

In [166]:
##build concatenated X file for fitting
def build_data_file(all_data,frame):
    X = []
    for run in range(1,4):
        #get indices of events of interest and mask data
        event_indices = np.array([False]*all_data['run'+str(run)].shape[0])
        indices = np.array(timing[timing['run']==run]['tr_index'].values,dtype=int) + frame
        event_indices[indices] = True
        fmri_masked = all_data['run'+str(run)][event_indices,:]

        X.extend(fmri_masked)

    X = np.array(X) #convert to numpy array
    return X

In [182]:
def run_decoding(in_tuple):
    
    sub, roi, exp = in_tuple
    
    all_data = load_data(sub,roi)
    
    scores = []
    for frame in frames:
        X = build_data_file(all_data,frame)

        #set up classifier
        classifier = LogisticRegression(penalty = 'l2', C=1.)
        cv = LeaveOneLabelOut(timing['run'].values-1)

        #convert labels to TF
        cond_map = {'serial':True,'single':False}
        target = map(lambda x: cond_map[x], timing['condition'].values)

        res = cross_val_score(classifier, X, target, cv=cv, scoring = 'accuracy')

        scores.append(np.mean(res))
    

In [8]:
in_tuple = []
for sub in sub_list:
    for exp in exp_list:
        for roi in all_rois:
            in_tuple.append((sub,roi,exp))

In [ ]:
pool = multiprocessing.Pool(processes = 8)
pool.map(crop_sim_ser,in_tuple)
pool.terminate()
pool.join()